In [1]:
# 初始化Spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

sc = SparkContext()
spark = SparkSession(sparkContext=sc)


# 弹性分布式数据集RDD

## RDD的概念

RDD(Resilient Distributed Datasets)，**弹性分布式数据集**，是spark中最基本的数据抽象，它代表一个不可变、可分区、里面的元素可并行计算的集合。

RDD具有数据流模型的特点：自动容错、位置感知性调度和可伸缩性。

RDD允许用户在执行多个查询时显式地将工作集缓存在内存中，后续的插叙能够重用工作集，这极大地提高了查询速度。

## RDD的属性

1. 分区（Partition），即数据集的基本组成单位。对于RDD来说，每个分区都会被一个计算任务处理，并决定并行计算的粒度。用户可以在创建RDD时指定RDD的分区个数，如果没有指定，那么就会采用默认值。默认值就是程序所分配到的CPU Core的数目。（注：源代码中部分地方使用分片Slice，是为了保存向后兼容性）

2. 一个计算每个分区的函数。Spark中RDD的计算是以分区为单位的，每个RDD都会实现compute函数以达到这个目的。compute函数会对迭代器进行复合，不需要保存每次计算的结果。

3. RDD之间的依赖关系。RDD的每次转换都会生成一个新的RDD，所以RDD之间就会形成类似于流水线一样的前后依赖关系。在部分分区数据丢失时，Spark可以通过这个依赖关系重新计算丢失的分区数据，而不是对RDD的所有分区进行重新计算。

4. 一个Partitioner，即RDD的分区函数。当前Spark中实现了两种类型的分区函数，一个是基于哈希的HashPartitioner，另外一个是基于范围的RangePartitioner。只有对于于key-value的RDD，才会有Partitioner，非key-value的RDD的Parititioner的值是None。Partitioner函数不但决定了RDD本身的分区数量，也决定了parent RDD Shuffle输出时的分区数量。

5. 一个列表，存储存取每个Partition的优先位置（preferred location）。对于一个HDFS文件来说，这个列表保存的就是每个Partition所在的块的位置。按照“移动数据不如移动计算”的理念，Spark在进行任务调度的时候，会尽可能地将计算任务分配到其所要处理数据块的存储位置。

![WordCount图解RDD](../../images/WordCount图解RDD.png)

其中hello.txt

![hello.txt](../../images/hello.txt.png)


## RDD创建方式

创建RDD的方法有两种：
- `并行化`驱动程序中的现有集合
- `引用外部存储系统`（例如共享文件系统，HDFS，HBase或提供Hadoop InputFormat的任何数据源）中的数据集。

### 并行化

`SparkContext.parallelize`方法读取可迭代对象或集合(如:`dict`,`list`,`tuple`,`set`)，转换为可以并行计算的分布式数据集。

In [2]:
# from a list
rdd = sc.parallelize([1,2,3])
rdd.collect()

[1, 2, 3]

In [3]:
# from a tuple
rdd = sc.parallelize(('cat', 'dog', 'fish'))
rdd.collect()

['cat', 'dog', 'fish']

In [4]:
# from a list of tuple
list_t = [('cat', 'dog', 'fish'), ('orange', 'apple')]
rdd = sc.parallelize(list_t)
rdd.collect()

[('cat', 'dog', 'fish'), ('orange', 'apple')]

In [5]:
# from a set
s = {'cat', 'dog', 'fish', 'cat', 'dog', 'dog'}
rdd = sc.parallelize(s)
rdd.collect()

['dog', 'fish', 'cat']

In [6]:
# 对于`dict`, 仅用keys构建RDD
d = {
    'a': 100,
    'b': 200,
    'c': 300
}
rdd = sc.parallelize(d)
rdd.collect()

['a', 'b', 'c']

### 引用外部数据集

PySpark可以从Hadoop支持的任何存储源创建分布式数据集，包括本地文件系统，HDFS，Cassandra，HBase，Amazon S3等。Spark支持文本文件，SequenceFiles和任何其他Hadoop InputFormat。

`SparkContext.textFile`读取文件的URI（本地路径，或一个hdfs://，s3a://等URI），返回字符串RDD，其中一个字符串元素对应文件的一行。


In [7]:
# 读取文件
rdd = sc.textFile('../../data/mtcars.csv')
rdd.take(5)

[',mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb',
 'Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4',
 'Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4',
 'Datsun 710,22.8,4,108,93,3.85,2.32,18.61,1,1,4,1',
 'Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1']

In [8]:
# 支持输入目录
rdd = sc.textFile('../../data/titanic/')
rdd.take(5)

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S']

In [9]:
# 支持输入文件通配符
rdd = sc.textFile('../../data/*.csv')
rdd.take(5)

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S']

除文本文件外，Spark的Python API还支持其他几种数据格式：

- `SparkContext.wholeTextFiles`读取包含多个小文本文件的目录，并将每个小文本文件作为（文件名，内容）对返回。相比textFile，会在每个文件的每一行返回一条记录。

- `RDD.saveAsPickleFile`和`SparkContext.pickleFile`支持将RDD保存为由pickle序列化的Python对象组成的简单格式。可批处理pickle序列化，默认批处理大小为10。

- SequenceFile和Hadoop输入/输出格式


In [10]:
rdd = sc.wholeTextFiles("../../data")
rdd.first()

('file:/mnt/Workspace/spark/_tutorial/learn-spark-with-python/data/twitter.txt',
 'Fresh install of XP on new computer. Sweet relief! fuck vista\t1018769417\t1.0\nWell. Now I know where to go when I want my knives. #ChiChevySXSW http://post.ly/RvDl\t10284216536\t1.0\n"Literally six weeks before I can take off ""SSC Chair"" off my email. Its like the torturous 4th mile before everything stops hurting."\t10298589026\t1.0\nMitsubishi i MiEV - Wikipedia, the free encyclopedia - http://goo.gl/xipe Cutest car ever!\t109017669432377344\t1.0\n\'Cheap Eats in SLP\' - http://t.co/4w8gRp7\t109642968603963392\t1.0\nTeenage Mutant Ninja Turtle art is never a bad thing... http://bit.ly/aDMHyW\t10995492579\t1.0\nNew demographic survey of online video viewers: http://bit.ly/cx8b7I via @KellyOlexa\t11713360136\t1.0\nhi all - i\'m going to be tweeting things lookstat at the @lookstat twitter account. please follow me there\t1208319583\t1.0\nHoly carp, no. That movie will seriously suffer for it. RT @Mou

## RDD操作

RDD支持两种类型的操作：`转换`（transformations，从现有数据集创建新数据集）和`动作`（actions，在对数据集执行计算后，将值返回给驱动程序）。

Spark中的所有转换都是惰性的，因为它们不会立即计算出结果。相反，他们只记得应用于某些基本数据集（例如文件）的转换。仅当动作要求将结果返回给驱动程序时才计算转换。这种设计使Spark可以更高效地运行。例如，我们可以意识到，通过创建的数据集map将用于中，reduce并且仅将结果返回reduce给驱动程序，而不是将较大的映射数据集返回给驱动程序。

默认情况下，每次在其上执行动作时，都可能会重新计算每个转换后的RDD。但也可使用persist（或cache）方法将RDD保留在内存中，在这种情况下，Spark会将元素保留在集群中，以便下次查询时可以更快地进行访问。还支持将RDD持久保存在磁盘上，或在多个节点之间复制。

### 传lambda表达式

In [11]:
# 统计字符数
lines = sc.textFile("../../data/twitter.txt")
lineLengths = lines.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)
print(totalLength)

# 将RDD保存在内存中
lineLengths.persist()
totalLength = lineLengths.reduce(lambda a, b: a + b)
print(totalLength)


1087
1087


### 传函数

In [12]:
# 统计字数
def word_count(s):
    words = s.split(" ")
    return len(words)

wordCnts = lines.map(word_count)
wordCnts.reduce(lambda a, b: a + b)

151

### 传类方法或成员

In [13]:
# 注意：尽量不要传对象的成员或方法，因为如果引用某个对象的成员或方法，则整个对象都要传到集群中，如下
class MyClass(object):
    def __init__(self):
        self.field = "Hello"
    def func(self, s):
        return s
    def doStuff(self, rdd):
        return rdd.map(self.func)
    def doStuff2(self, rdd):
        return rdd.map(lambda s: self.field + s)
        
# 为避免上述问题，可将需要引用的成员或方法复制为局部变量再传入
class MyClass2(object):
    def __init__(self):
        self.field = "Hello"
    def func(self, s):
        return s
    def doStuff(self, rdd):
        _func = self.func
        return rdd.map(_func)
    def doStuff2(self, rdd):
        _filed = self.field
        return rdd.map(lambda s: _field + s)

### 闭包

如果要传闭包函数，则必须要了解跨集群执行代码时的变量和方法的范围和生命周期。

下述示例为计算RDD元素之和的代码。

In [14]:
counter = 0
rdd = sc.parallelize([1,2,3,4])

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x
rdd.foreach(increment_counter)

print("Counter value: ", counter)

Counter value:  0


Spark执行RDD操作会将任务分解到多个执行程序，而每个执行程序上的闭包的变量都是一个副本，因此在程序中的counter最终还是0。

通常，闭包不应改变某些全局状态。Spark不定义或保证从闭包外部引用的对象的改变行为。某些执行此操作的代码可能会在本地模式下工作，但这只是偶然的情况，此类代码在分布式模式下将无法按预期运行。如果需要某些全局汇总，可使用累加器`Accumulator`。

另外，关于打印RDD元素也要注意这个问题，在单台机器上使用`rdd.foreach(print)`或`rdd.map(print)`是可行的，但在集群模式下应该使用`collect`方法，

In [15]:
rdd = sc.parallelize([1,2,3,4])
rdd.collect()

[1, 2, 3, 4]

### 使用键值对

In [16]:
# 统计字符数
lines = sc.textFile("../../data/twitter.txt")
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)
counts.sortByKey().collect()

[('"Did I really need to learn ""I bought a box and put in it things"" in arabic? This is the most random book ever."\t12358025545\t1.0',
  1),
 ('"Literally six weeks before I can take off ""SSC Chair"" off my email. Its like the torturous 4th mile before everything stops hurting."\t10298589026\t1.0',
  1),
 ("'Cheap Eats in SLP' - http://t.co/4w8gRp7\t109642968603963392\t1.0", 1),
 ('Fresh install of XP on new computer. Sweet relief! fuck vista\t1018769417\t1.0',
  1),
 ('Holy carp, no. That movie will seriously suffer for it. RT @MouseInfo: Anyone excited for The Little Mermaid in 3D?\t121330835726155776\t1.0',
  1),
 ('Mitsubishi i MiEV - Wikipedia, the free encyclopedia - http://goo.gl/xipe Cutest car ever!\t109017669432377344\t1.0',
  1),
 ('New demographic survey of online video viewers: http://bit.ly/cx8b7I via @KellyOlexa\t11713360136\t1.0',
  1),
 ('Teenage Mutant Ninja Turtle art is never a bad thing... http://bit.ly/aDMHyW\t10995492579\t1.0',
  1),
 ('Well. Now I know where

### 常见转换

- **map**(*func*) - 返回一个新的RDD，该RDD是由将源数据集的每个元素经过函数*func*计算后返回值组成。

- **filter**(*func*) - 返回一个新的RDD，该RDD由经过*func*函数计算后返回值为true的输入元素组成

- **flatMap**(*func*) - 类似于map，但是每一个输入元素可以被映射为0或多个输出元素（所以*func*应该返回一个序列，而不是单一元素）

- **mapPartitions**(*func*) - 类似于map，但独立地在RDD的每一个分片上运行，因此在类型为T的RDD上运行时，*func*的函数类型必须是Iterator&lt;T> => Iterator&lt;U>

- **mapPartitionsWithIndex**(*func*) - 类似于mapPartitions，但*func*带有一个整数参数表示分片的索引值，因此在类型为T的RDD上运行时，*func*的函数类型必须是(Int, Interator&lt;T>) => Iterator&lt;U>

- **sample**(*withReplacement*, *fraction*, *seed*) - 根据*fraction*指定的比例对数据进行采样，可以选择是否使用随机数进行替换，*seed*用于指定随机数生成器种子

- **union**(*otherDataset*) - 对源RDD和参数RDD求并集后返回一个新的RDD

- **intersection**(*otherDataset*) - 对源RDD和参数RDD求交集后返回一个新的RDD

- **distinct**(\[*numPartitions*\])) - 对源RDD进行去重后返回一个新的RDD

- **groupByKey**(\[*numPartitions*\]) - 在(K,V)对的RDD上调用，返回(K, Iterator&lt;V>)对的RDD。 **注意:**(1)如果要分组以便对每个键执行汇总（例如求和或平均值），则使用`reduceByKey`或`aggregateByKey`性能会更好。(2)默认情况下，输出中的并行度取决于父RDD的分区数。可以传递一个可选numPartitions参数来设置不同数量的任务。

- **reduceByKey**(*func*, \[*numPartitions*\]) - 在(K,V)对的RDD上调用，返回(K,V)对的RDD，其中每个键的值由指定的reduce函数*func*汇总得到，函数类型必须为(V,V) => V。与groupByKey类似，reduce任务的个数可以通过第二个可选的参数来设置

- **aggregateByKey**(*zeroValue*)(*seqOp*, *combOp*, \[*numPartitions*\]) - 在(K,V)对的RDD上调用，返回(K,U)对的RDD，其中每个键的值由给定的组合函数*combOp*加中性"零"值进行汇总。允许输入值类型与汇合值类型不同，采用此方法可避免一些不必要的分配。例如：aggregateByKey(0)(_+_,_+_) 对k/y的RDD进行操作

- **sortByKey**(\[*ascending*\], \[*numPartitions*\]) - 在(K,V)对的RDD上调用，返回按K排序的(K,V)对RDD。必须实现Ordered接口。

- **sortBy**(*func*,\[*ascending*\], \[*numPartitions*\]) - 与sortByKey类似，但是更灵活。第一个参数是根据什么排序；第二个是否升序；第三个排序后分区数，默认与原RDD一样。

- **join**(*otherDataset*, \[*numPartitions*\]) - 在(K,V)和(K,W)的RDD上调用，返回(K,(V,W))的RDD，其中每个键都有所有成对的元素，相当于内连接(求交集)。外连接通过外连接leftOuterJoin，rightOuterJoin和fullOuterJoin支持。

- **leftOuterJoin** - leftOuterJoin类似于SQL中的左外关联left outer join，返回结果以前面的RDD为主，关联不上的记录为空。只能用于两个RDD之间的关联，如果要多个RDD关联，多关联几次即可。

- **rightOuterJoin** - rightOuterJoin类似于SQL中的有外关联right outer join，返回结果以参数中的RDD为主，关联不上的记录为空。只能用于两个RDD之间的关联，如果要多个RDD关联，多关联几次即可

- **cogroup**(*otherDataset*, \[*numPartitions*\]) - 在(K,V)和(K,W)的RDD上调用，返回一个(K,(Iterable&lt;V>,Iterable<W&gt))类型的RDD。此操作也称`groupWith`

- **cartesian**(*otherDataset*) - 在类型T和U的RDD上调用时，返回（T，U）所有元素对（笛卡尔积）的RDD

- **pipe**(*command*, \[*envVars*\]) - 通过管道RDD的每个分区传递给Shell命令（例如Perl或bash脚本）。将RDD元素写入进程的stdin，并将输出到其stdout的行作为字符串的RDD返回。

- **coalesce**(*numPartitions*) - 重新分区，第一个参数是要分多少区，第二个参数是否shuffle，默认false，少分区变多分区-true，多分区变少分区-false

- **repartition**(*numPartitions*) - 重新分区，必须shuffle，参数是要分多少区

- **repartitionAndSortWithinPartitions**(*partitioner*) - 根据给定的分区程序*partitioner*对RDD重新分区，并在每个结果分区中，按其键对记录进行排序。这比`repartition`在每个分区内调用然后排序更为有效，因为它可以将排序沉入洗牌机制。

- **foldByKey**(*zeroValue*)(*seqOp*) - 该函数用于K/V做折叠，合并处理。与aggregate类似，第一个括号的参数应用于每个V值  第二括号函数是聚合例如：_+_

- **combineByKey** - 合并相同的key的值。rdd1.combineByKey(x => x, (a: Int, b: Int) => a + b, (m: Int, n: Int) => m + n)

- **partitionBy**(*partitioner*) - 对RDD进行分区。partitioner是分区器 例如new HashPartition(2

- **cache**|**persist**：RDD缓存，可以避免重复计算从而减少时间。区别：cache内部调用了persist算子，cache默认就一个缓存级别MEMORY-ONLY ，而persist则可以选择缓存级别

- **subtract**(*rdd*)- 返回前RDD元素不在后*rdd*的RDD

- **subtractByKey**(*otherRDD*) - substractByKey和subtract类似，只不过这里是针对K的，返回在主RDD中出现，并且不在*otherRDD*中出现的元素


### 常见动作
- **reduce**(*func*) - 使用函数*func*(该函数接受两个参数并返回一个)来汇总数据集的元素。该函数应该是可交换的和关联的，以便可以并行正确地计算它。

- **collect**() - 在驱动程序中，以数组的形式返回数据集的所有元素

- **count**() - 返回RDD的元素个数

- **first**() - 返回RDD的第一个元素(类似于take(1))

- **take**(*n*) - 返回一个由数据集的前*n*个元素组成的数组

- **takeSample**(*withReplacement*,*num*, \[*seed*\]) - 返回一个数组，该数组由从数据集中随机采样的num个元素组成，可以选择是否用随机数替换不足的部分，seed用于指定随机数生成器种子

- **takeOrdered**(*n*, \[*ordering*\]) - 使用自然顺序或自定义比较器*ordering*返回RDD的前n个元素。

- **saveAsTextFile**(*path*) - 将数据集的元素以textfile的形式保存到HDFS文件系统或者其他支持的文件系统，对于每个元素，Spark将会调用toString方法，将它装换为文件中的一行文本

- **saveAsSequenceFile**(*path*) （Java和Scala）- 在本地文件系统，HDFS或任何其他Hadoop支持的文件系统的给定路径中，将数据集的元素作为Hadoop SequenceFile写入。在实现Hadoop的Writable接口的键值对的RDD上可用。在Scala中，它也可用于隐式转换为Writable的类型（Spark包括对基本类型（如Int，Double，String等）的转换）。

- **saveAsObjectFile**(*path*) （Java和Scala）- 使用Java序列化以简单的格式编写数据集的元素，然后可以使用`SparkContext.objectFile()`进行加载。

- **countByKey**() - 针对(K,V)类型的RDD，返回一个(K,Int)的map，表示每一个key对应的元素个数。

- **foreach**(*func*) - 在数据集的每一个元素上，运行函数*func*进行更新。

- **foreachPartition** - 


- **aggregate** - 先对分区进行操作，在总体操作

- **reduceByKeyLocally** - 

- **lookup** - 

- **top** - 

- **fold** - 



 